## Artificial Intelligence Assignment 3
## No3. CNN5 Log Likelihood Loss
## 20132651 Sungjae Lee

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [3]:
## 새로운 Loss Function 으로 Negative Log Likelihood Loss 를 사용합니다.
## torch.nn 에서 구현되어 있으며, 기존의 Loss 를 nn.NLLLoss 로 변환하면 사용 가능합니다.

import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### (3) 화면 출력 확인 및 학습이 되고 있는지 서술

In [4]:
## NLL Loss 를 사용할 때 주의해야 할 점은, 출력에 LogSoftmax 를 적용한 다음
## Loss 를 계산해야 한다는 것입니다. 이를 구현하기 위해 nn.LogSoftmax 를 m 으로 정의하고,
## loss 를 계산하는 criterion(outputs, labels) 에서 outputs 에 m 함수를 적용시킵니다.
## 이로써 loss 가 음수가 아닌 양수의 범위에서 가장 작은 값으로 수렴하도록 학습이 진행됩니다.

m = nn.LogSoftmax()
for epoch in range(2):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(m(outputs), labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss : %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

## 학습의 결과는 이전과 크게 다르지 않으며, 최저 loss 는 1.275 가 출력되었습니다.
## Optimizer 나 batch norm 을 사용하지 않았기 때문인지, 초반의 loss 감소가 더딘점이 눈에 띄었습니다.

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


[1,  1000] loss : 2.299
[1,  2000] loss : 2.123
[1,  3000] loss : 1.911
[1,  4000] loss : 1.782
[1,  5000] loss : 1.680
[1,  6000] loss : 1.610
[1,  7000] loss : 1.570
[1,  8000] loss : 1.519
[1,  9000] loss : 1.555
[1, 10000] loss : 1.503
[1, 11000] loss : 1.469
[1, 12000] loss : 1.454
[2,  1000] loss : 1.403
[2,  2000] loss : 1.408
[2,  3000] loss : 1.369
[2,  4000] loss : 1.387
[2,  5000] loss : 1.338
[2,  6000] loss : 1.321
[2,  7000] loss : 1.307
[2,  8000] loss : 1.293
[2,  9000] loss : 1.343
[2, 10000] loss : 1.295
[2, 11000] loss : 1.271
[2, 12000] loss : 1.275
Finished Training


### (6) 화면 출력 확인 및 일반화 성능 서술

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images :%d %%'%(100 * correct / total))

## 테스트 데이터에 대한 

Accuracy of the network on the 10000 test images :52 %


In [6]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

### (7) 화면 출력 확인 및 부류별 분류기의 성능 서술

In [7]:
for i in range(10):
    print('Accuracy of %5s: %2d %%'%(classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane: 45 %
Accuracy of   car: 81 %
Accuracy of  bird: 22 %
Accuracy of   cat: 35 %
Accuracy of  deer: 48 %
Accuracy of   dog: 26 %
Accuracy of  frog: 51 %
Accuracy of horse: 82 %
Accuracy of  ship: 76 %
Accuracy of truck: 49 %
